In [ ]:
import os
import requests
import zipfile
from tqdm import tqdm
import shutil
import json

def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    with open(destination, 'wb') as file, tqdm(
            desc=os.path.basename(destination),
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
        for data in response.iter_content(block_size):
            size = file.write(data)
            bar.update(size)

def download_coco_validation_set(root_dir='./coco_data'):
    os.makedirs(root_dir, exist_ok=True)

    # Check if already downloaded
    if os.path.exists(os.path.join(root_dir, 'val2017')) and \
       os.path.exists(os.path.join(root_dir, 'annotations')):
        print("COCO validation set already exists!")
        return os.path.join(root_dir, 'val2017'), os.path.join(root_dir, 'annotations/instances_val2017.json')

    # URLs for downloading
    val_images_url = "http://images.cocodataset.org/zips/val2017.zip"
    annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

    # Download validation images
    print("Downloading COCO validation images (5000 images)...")
    val_zip_path = os.path.join(root_dir, "val2017.zip")
    download_file(val_images_url, val_zip_path)

    # Download annotations
    print("ddownloading COCO annotations...")
    ann_zip_path = os.path.join(root_dir, "annotations_trainval2017.zip")
    download_file(annotations_url, ann_zip_path)

    # Extract validation images
    print("extracting validation images...")
    with zipfile.ZipFile(val_zip_path, 'r') as zip_ref:
        zip_ref.extractall(root_dir)

    # Extract annotations
    print("extracting annotations...")
    with zipfile.ZipFile(ann_zip_path, 'r') as zip_ref:
        zip_ref.extractall(root_dir)

    # Clean up zip files
    os.remove(val_zip_path)
    os.remove(ann_zip_path)

    print("COCO validation set downloaded successfully!")
    return os.path.join(root_dir, 'val2017'), os.path.join(root_dir, 'annotations/instances_val2017.json')

def verify_coco_dataset(images_dir, annotations_file):
    # Check if images directory exists and has files
    if not os.path.exists(images_dir):
        print(f"Error: Images directory {images_dir} does not exist")
        return False

    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
    print(f"Found {len(image_files)} images in {images_dir}")

    # Check if annotations file exists and is valid JSON
    if not os.path.exists(annotations_file):
        print(f"Error: Annotations file {annotations_file} does not exist")
        return False

    try:
        with open(annotations_file, 'r') as f:
            annotations = json.load(f)
            num_images = len(annotations['images'])
            num_annotations = len(annotations['annotations'])
            print(f"Annotations file contains information for {num_images} images and {num_annotations} annotations")
    except Exception as e:
        print(f"Error reading annotations file: {e}")
        return False

    print("Dataset verification completed successfully!")
    return True

images_dir, annotations_file = download_coco_validation_set()

# Verify the dataset
verify_coco_dataset(images_dir, annotations_file)

print("\nDataset information:")
print(f"- Images directory: {images_dir}")
print(f"- Annotations file: {annotations_file}")
print("\nThe COCO validation set (5000 images) with annotations is ready for evaluation!")

val2017.zip: 100%|██████████| 778M/778M [00:18<00:00, 44.0MiB/s]


annotations_trainval2017.zip: 100%|██████████| 241M/241M [00:05<00:00, 49.2MiB/s]


Extracting validation images...
Extracting annotations...
COCO validation set downloaded successfully!
Found 5000 images in ./coco_data/val2017
Annotations file contains information for 5000 images and 36781 annotations
Dataset verification completed successfully!

Dataset information:
- Images directory: ./coco_data/val2017
- Annotations file: ./coco_data/annotations/instances_val2017.json

The COCO validation set (5000 images) with annotations is ready for evaluation!


In [ ]:
import torch
import torchvision
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from pycocotools.coco import COCO
import os
from PIL import Image
import numpy as np

class COCOEvalDataset(Dataset):
    """COCO dataset for evaluation with annotations"""
    def __init__(self, root, annFile,transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

        # Get categories
        self.categories = self.coco.loadCats(self.coco.getCatIds())
        self.categories.sort(key=lambda x: x['id'])
        self.category_ids = [category['id'] for category in self.categories]
        self.category_names = [category['name'] for category in self.categories]
    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        # Load image
        img_id = self.ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root, img_info['file_name'])
        img = Image.open(img_path).convert('RGB')

        # Apply transformations that match the pretrained models
        transform = T.Compose([
            T.ToTensor(),
            # T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image = transform(img)

        # Get annotations
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for ann in anns:
            x, y, w, h = ann['bbox']
            # Convert to [x1, y1, x2, y2] format
            boxes.append([x, y, (x + w), (y + h)])

            # Map COCO category_id to model index
            # Handle case where category might not be in our mapping
            cat_id = ann['category_id']
            if cat_id in self.category_ids:
                labels.append(cat_id)
            else:
                # Skip annotations with unknown categories
                continue

            areas.append(ann['area'])
            iscrowd.append(ann['iscrowd'])

        # Convert to tensors
        if boxes:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            areas = torch.as_tensor(areas, dtype=torch.float32)
            iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        else:
            # Handle images with no annotations
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            areas = torch.zeros((0,), dtype=torch.float32)
            iscrowd = torch.zeros((0,), dtype=torch.int64)

        # Create target dict
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id]),
            'area': areas,
            'iscrowd': iscrowd
        }

        return image, target

def create_coco_loader(images_dir, annotations_file, batch_size=1, num_workers=4):
    """Create a data loader for COCO evaluation"""
    # Define transforms
    transform = T.Compose([
        # T.Resize((300, 300)),  # Resize to 300x300
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create dataset
    dataset = COCOEvalDataset(
        root=images_dir,
        annFile=annotations_file,
        transform=transform
    )

    # Create data loader
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=lambda x: x  # Keep variable sized tensors
    )


    print(f"Created data loader with {len(dataset)} images for evaluation")
    return loader, dataset

images_dir = "./coco_data/val2017"
annotations_file = "./coco_data/annotations/instances_val2017.json"

# Create data loader
loader, dataset = create_coco_loader(images_dir, annotations_file)

# Print some dataset statistics
print(f"Number of images: {len(dataset)}")
print(f"Number of categories: {len(dataset.categories)}")

# Display some category names
print("\nSome COCO categories:")
for i, name in enumerate(dataset.category_names):
    print(f"  {i+1}. {name}")

print("\nDataset is ready for evaluation!")

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!
Created data loader with 5000 images for evaluation
Number of images: 5000
Number of categories: 80

Some COCO categories:
  1. person
  2. bicycle
  3. car
  4. motorcycle
  5. airplane
  6. bus
  7. train
  8. truck
  9. boat
  10. traffic light
  11. fire hydrant
  12. stop sign
  13. parking meter
  14. bench
  15. bird
  16. cat
  17. dog
  18. horse
  19. sheep
  20. cow
  21. elephant
  22. bear
  23. zebra
  24. giraffe
  25. backpack
  26. umbrella
  27. handbag
  28. tie
  29. suitcase
  30. frisbee
  31. skis
  32. snowboard
  33. sports ball
  34. kite
  35. baseball bat
  36. baseball glove
  37. skateboard
  38. surfboard
  39. tennis racket
  40. bottle
  41. wine glass
  42. cup
  43. fork
  44. knife
  45. spoon
  46. bowl
  47. banana
  48. apple
  49. sandwich
  50. orange
  51. broccoli
  52. carrot
  53. hot dog
  54. pizza
  55. donut
  56. cake
  57. chair
  58. couch
  59. potted 

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# from torchvision.io.image import decode_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights,FasterRCNN_MobileNet_V3_Large_FPN_Weights,retinanet_resnet50_fpn_v2,RetinaNet_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
# from coco_loader import create_coco_loader
import torch
from tqdm import tqdm
import numpy as np
import json
from collections import defaultdict
import torchvision
import torch
import torchvision
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os
import json
# from coco_loader import create_coco_loader
from PIL import Image
import zipfile
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.ops import nms

def calculate_iou(box1, box2):
    """
    Calculate IoU between two bounding boxes
    Box format: [x1, y1, x2, y2]
    """
    # Calculate intersection area
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area if union_area > 0 else 0.0

def convert_to_xywh(box):
    """Convert box from [x1, y1, x2, y2] to [x, y, width, height]"""
    return [box[0], box[1], box[2] - box[0], box[3] - box[1]]

def convert_to_xyxy(box):
    """Convert box from [x, y, width, height] to [x1, y1, x2, y2]"""
    return [box[0], box[1], box[0] + box[2], box[1] + box[3]]

def compute_ap(precision, recall):
    """Compute Average Precision using 11-point interpolation"""
    ap = 0.0
    for t in np.arange(0.0, 1.1, 0.1):
        if np.sum(recall >= t) == 0:
            p = 0
        else:
            p = np.max(precision[recall >= t])
        ap += p / 11.0
    return ap

def evaluate_detections(predictions, ground_truth, iou_threshold=0.5, max_dets=100):
    # Group predictions and ground truth by image_id and category_id
    pred_by_img = defaultdict(list)
    gt_by_img = defaultdict(lambda: defaultdict(list))

    # Get all category ids
    category_ids = set()

    # Process predictions
    for pred in predictions:
        img_id = pred['image_id']
        cat_id = pred['category_id']
        pred_by_img[img_id].append(pred)
        category_ids.add(cat_id)

    # Process ground truth
    for gt in ground_truth:
        img_id = gt['image_id']
        cat_id = gt['category_id']
        gt_by_img[img_id][cat_id].append(gt)
        category_ids.add(cat_id)

    # Metrics storage
    metrics = {
        'precision': {},
        'recall': {},
        'ap': {},
        'f1_score': {}
    }

    # Calculate metrics for each category
    for cat_id in category_ids:
        true_positives = []
        false_positives = []
        scores = []
        num_gt = 0

        # Count total ground truth for this category
        for img_id in gt_by_img:
            num_gt += len(gt_by_img[img_id].get(cat_id, []))

        # Process each image
        for img_id in pred_by_img:
            # Get predictions for this image
            img_preds = [p for p in pred_by_img[img_id] if p['category_id'] == cat_id]

            # Sort predictions by score in descending order
            img_preds = sorted(img_preds, key=lambda x: x['score'], reverse=True)

            # Limit number of detections
            img_preds = img_preds[:max_dets]

            # Get ground truth for this image and category
            img_gts = gt_by_img[img_id].get(cat_id, [])

            # Mark each ground truth as matched or not
            matched_gt = [False] * len(img_gts)

            # Check each prediction
            for pred in img_preds:
                pred_bbox = convert_to_xyxy(pred['bbox'])
                pred_score = pred['score']

                best_iou = 0
                best_gt_idx = -1

                # Find best matching ground truth
                for gt_idx, gt in enumerate(img_gts):
                    if matched_gt[gt_idx]:
                        continue

                    gt_bbox = convert_to_xyxy(gt['bbox'])
                    iou = calculate_iou(pred_bbox, gt_bbox)

                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = gt_idx

                # Check if we have a match
                if best_iou >= iou_threshold and best_gt_idx >= 0:
                    true_positives.append(1)
                    false_positives.append(0)
                    matched_gt[best_gt_idx] = True
                else:
                    true_positives.append(0)
                    false_positives.append(1)

                scores.append(pred_score)

        # Sort by score
        inds = np.argsort(scores)[::-1]
        true_positives = np.array(true_positives)[inds]
        false_positives = np.array(false_positives)[inds]

        # Compute cumulative sum
        tp_cumsum = np.cumsum(true_positives)
        fp_cumsum = np.cumsum(false_positives)

        # Compute precision and recall
        precision = tp_cumsum / (tp_cumsum + fp_cumsum + np.finfo(float).eps)
        recall = tp_cumsum / (num_gt + np.finfo(float).eps)

        # Compute AP
        ap = compute_ap(precision, recall)

        # Store metrics
        metrics['precision'][cat_id] = precision
        metrics['recall'][cat_id] = recall
        metrics['ap'][cat_id] = ap

        # Compute F1 score (optional)
        if len(precision) > 0 and len(recall) > 0:
            f1 = 2 * precision * recall / (precision + recall + np.finfo(float).eps)
            metrics['f1_score'][cat_id] = np.max(f1)
        else:
            metrics['f1_score'][cat_id] = 0.0

    # Compute mAP
    metrics['mAP'] = np.mean([metrics['ap'][cat_id] for cat_id in metrics['ap']])

    return metrics

def main():
    # Initialize model

    # model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1)
    # weights= FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1
    # weights = FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
    # model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.5)
    model=retinanet_resnet50_fpn_v2(weights=RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1)
    weights=RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1
    model.eval()


    # Initialize preprocessing
    preprocess = weights.transforms()

    # Load COCO dataset
    images_dir = "./coco_data/val2017"
    annotations_file = "./coco_data/annotations/instances_val2017.json"
    loader, dataset = create_coco_loader(images_dir, annotations_file)

    # Run inference
    results = []
    ground_truth = []
    img,target=dataset[0]
    visualize_detections(model,dataset)

    print("Running inference...")
    with torch.no_grad():
        for i, (img, target) in enumerate(tqdm(dataset)):
            if i >= 5000:  # Limit to first 100 images for testing
                break

            image_id = target['image_id'].item()

            # Add ground truth for this image
            for obj_idx in range(len(target['boxes'])):
                gt = {
                    'image_id': image_id,
                    'category_id': target['labels'][obj_idx].item(),
                    'bbox': convert_to_xywh(target['boxes'][obj_idx].tolist()),
                    'iscrowd': 0
                }
                ground_truth.append(gt)

            # Run model
            batch = [preprocess(img.to('cuda'))]
            prediction = model(batch)[0]

            # Extract predictions
            boxes = prediction['boxes'].cpu().numpy()
            scores = prediction['scores'].cpu().numpy()
            labels = prediction['labels'].cpu().numpy()

            # Convert predictions to COCO format
            for box, score, label in zip(boxes, scores, labels):
                if score >= 0.05:  # Minimum score threshold
                    # print(score)
                    x1, y1, x2, y2 = box.tolist()
                    coco_box = [x1, y1, x2 - x1, y2 - y1]  # [x, y, width, height]

                    category_id = label.item()

                    results.append({
                        'image_id': image_id,
                        'category_id': category_id,
                        'bbox': coco_box,
                        'score': float(score)
                    })


    # Save results to file
    with open('custom_results.json', 'w') as f:
        json.dump(results, f)

    # Run custom evaluation
    print("Running custom evaluation...")
    metrics = evaluate_detections(results, ground_truth)

    # Print results
    print("\nEvaluation Results:")
    print(f"mAP @ IoU={0.5}: {metrics['mAP']:.4f}")

    # Print AP for each category
    print("\nAP by category:")
    for cat_id in sorted(metrics['ap'].keys()):
        print(f"Category {cat_id}: {metrics['ap'][cat_id]:.4f}")

    # Optional: Plot precision-recall curves
    try:
        import matplotlib.pyplot as plt

        # Plot precision-recall curve for first 5 categories
        plt.figure(figsize=(10, 8))

        for i, cat_id in enumerate(sorted(list(metrics['precision'].keys()))[:5]):
            if len(metrics['precision'][cat_id]) > 0 and len(metrics['recall'][cat_id]) > 0:
                plt.plot(
                    metrics['recall'][cat_id],
                    metrics['precision'][cat_id],
                    label=f'Category {cat_id} (AP: {metrics["ap"][cat_id]:.4f})'
                )

        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves')
        plt.legend()
        plt.grid()
        plt.savefig('precision_recall_curves.png')
        plt.close()
        print("Saved precision-recall curves to 'precision_recall_curves.png'")
    except ImportError:
        print("Matplotlib not installed. Skipping precision-recall curve plotting.")



def visualize_detections(model, dataset, indices=None, n_examples=5, confidence_threshold=0.5):
    model.eval()
    device = next(model.parameters()).device

    indices = np.random.choice(len(dataset), n_examples, replace=False)

    plt.figure(figsize=(15, n_examples * 5))

    for i, idx in enumerate(indices):
        img, target = dataset[idx]

        # Create a copy of the image for drawing
        img_np = img.permute(1, 2, 0).cpu().numpy()
        # Denormalize
        img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img_np = np.clip(img_np, 0, 1)

        # Make prediction
        with torch.no_grad():
            prediction = model([img.to(device)])[0]

        # Plot the image
        plt.subplot(n_examples, 1, i + 1)
        plt.imshow(img_np)


        for box, label in zip(target['boxes'], target['labels']):
            x, y, w, h = box[0].item(), box[1].item(), box[2].item() - box[0].item(), box[3].item() - box[1].item()
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='g', facecolor='none')
            plt.gca().add_patch(rect)
            category_id = label.item()
            category_name = next(cat['name'] for cat in dataset.categories if cat['id'] == category_id)
            plt.text(x, y-5, category_name, color='g', fontsize=10, backgroundcolor='w')


        for box, label, score in zip(prediction['boxes'], prediction['labels'], prediction['scores']):
            if score >= confidence_threshold:
                x, y, w, h = box[0].item(), box[1].item(), box[2].item() - box[0].item(), box[3].item() - box[1].item()
                rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
                plt.gca().add_patch(rect)


                idx_to_cat = {v: k for k, v in dataset.category_id_to_idx.items()}
                category_id = label.item()
                category_name = next((cat['name'] for cat in dataset.categories if cat['id'] == category_id), "unknown")

                plt.text(x, y+h+15, f"{category_name}: {score:.2f}", color='r', fontsize=10, backgroundcolor='w')

        plt.axis('off')
        plt.title(f"Image {target['image_id'].item()}")

    plt.tight_layout()
    plt.savefig('coco_detection_examples.png')
    plt.close()
    print(f"Saved detection visualizations to coco_detection_examples.png")

torch.set_default_device('cuda')
main()

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:00<00:00, 170MB/s]


loading annotations into memory...
Done (t=0.75s)
creating index...
index created!
Created data loader with 5000 images for evaluation


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Saved detection visualizations to coco_detection_examples.png
Running inference...


100%|██████████| 5000/5000 [10:37<00:00,  7.84it/s]


Running custom evaluation...

Evaluation Results:
mAP @ IoU=0.5: 0.5978

AP by category:
Category 1: 0.7571
Category 2: 0.5649
Category 3: 0.6503
Category 4: 0.6962
Category 5: 0.8572
Category 6: 0.8100
Category 7: 0.8342
Category 8: 0.5587
Category 9: 0.5150
Category 10: 0.5399
Category 11: 0.8495
Category 13: 0.7079
Category 14: 0.6213
Category 15: 0.3814
Category 16: 0.5174
Category 17: 0.8812
Category 18: 0.8078
Category 19: 0.7882
Category 20: 0.7620
Category 21: 0.7566
Category 22: 0.8267
Category 23: 0.8949
Category 24: 0.8615
Category 25: 0.8691
Category 27: 0.3226
Category 28: 0.6113
Category 31: 0.3025
Category 32: 0.5225
Category 33: 0.5877
Category 34: 0.8174
Category 35: 0.4750
Category 36: 0.5419
Category 37: 0.6090
Category 38: 0.5920
Category 39: 0.5602
Category 40: 0.6384
Category 41: 0.7810
Category 42: 0.5864
Category 43: 0.7733
Category 44: 0.5520
Category 46: 0.5700
Category 47: 0.5830
Category 48: 0.5067
Category 49: 0.3002
Category 50: 0.3033
Category 51: 0.5643
